In [7]:
import os
import cv2
import torch
import torchvision.transforms as transforms
from PIL import Image
from models import shufflenetv2, nasnet_mobile_onfire
from tqdm import tqdm


In [8]:
def load_model(model_name, weight_path, device):
    if model_name == "shufflenetonfire":
        model = shufflenetv2.shufflenet_v2_x0_5(pretrained=False, layers=[4, 8, 4],
                                                output_channels=[24, 48, 96, 192, 64], num_classes=1)
    elif model_name == "nasnetonfire":
        model = nasnet_mobile_onfire.nasnetamobile(num_classes=1, pretrained=False)
    else:
        raise ValueError("Invalid model name. Choose 'shufflenetonfire' or 'nasnetonfire'.")
    model.load_state_dict(torch.load(weight_path, map_location=device))
    model.eval()
    model.to(device)
    return model

def get_transform(model_name):
    if model_name == 'shufflenetonfire':
        return transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])
    elif model_name == 'nasnetonfire':
        return transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

def preprocess_image(image_path, transform, device, rotation=0):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224), cv2.INTER_AREA)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    if rotation != 0:
        image = image.rotate(rotation)
    image = transform(image).float().unsqueeze(0).to(device)
    return image

def detect_fire(image_path, model, transform, device):
    # Get predictions for all rotations
    rotations = [0, 90, 180, 270]
    outputs = []
    
    for rotation in rotations:
        image_tensor = preprocess_image(image_path, transform, device, rotation)
        output = model(image_tensor)
        outputs.append(torch.sigmoid(output).item())
    
    # Take the minimum confidence score across all rotations
    # This makes it more sensitive to fire detection - if any rotation suggests fire, it will be detected
    min_output = min(outputs)
    print(min_output)
    prediction = 0 if min_output < 0.5 else 1
    return "Fire" if prediction == 0 else "No Fire"

In [9]:
# --- Set Parameters ---
model_name = "shufflenetonfire"            # or "nasnetonfire"
weight_path = "weights/shufflenet_ff.pt"      # adjust path if needed
folder_path = "val-data/fire"        # update folder path
ground_truth = "Fire"                      # set to "Fire" or "No Fire"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = load_model(model_name, weight_path, device)
transform = get_transform(model_name)

In [10]:
false_count = 0
total_images = 0
valid_extensions = (".jpg", ".jpeg", ".png", ".bmp")

# Iterate over files with a progress bar
for filename in tqdm(os.listdir(folder_path), desc="Processing images"):
    if filename.lower().endswith(valid_extensions):
        total_images += 1
        image_path = os.path.join(folder_path, filename)
        result = detect_fire(image_path, model, transform, device)
        # Count false positives (for "No Fire" ground truth) or false negatives (for "Fire")
        if (ground_truth == "No Fire" and result == "Fire") or (ground_truth == "Fire" and result == "No Fire"):
            false_count += 1

# Calculate correct classifications and accuracy
correct_count = total_images - false_count
accuracy = (correct_count / total_images * 100) if total_images > 0 else 0

print(f"Total images tested: {total_images}")
print(f"Correct classifications: {correct_count}")
if ground_truth == "No Fire":
    print(f"False Positives: {false_count}")
else:
    print(f"False Negatives: {false_count}")
print(f"Accuracy: {accuracy:.2f}%")







Processing images:   0%|          | 0/164 [00:00<?, ?it/s]

Processing images:   1%|          | 1/164 [00:00<00:45,  3.54it/s]

0.9995589852333069


Processing images:   1%|          | 2/164 [00:00<00:44,  3.63it/s]

0.0012975686695426702


...

0.9956510663032532
0.9998825788497925


Processing images:  99%|█████████▉| 163/164 [00:37<00:00,  4.85it/s]

0.9814722537994385


Processing images: 100%|██████████| 164/164 [00:37<00:00,  4.36it/s]

0.07908184826374054
Total images tested: 164
Correct classifications: 77
False Negatives: 87
Accuracy: 46.95%
